# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `clanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g., copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g., `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g., `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for some sequence containers (e.g., `vector` of the `std` namespace) and associative containers (e.g., `set`, `unordered_set` of the `std` namespace).
These template instantiations are done for various *C++* fundamental types (e.g., `int`, `unsigned long int`, `double`) and the `string` of the `std` namespace.
For ordered associative containers only the `std::less` comparator was used.
For the complete procedure refer to the `AutoWIG.py` file situed at the root of the **STL** [repository](https://github.com/StatisKit/STL).


We here aim at presenting how template libraries can be wrapped.
First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [ ]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [ ]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

In this notebook, we do not need to import **AutoWIG** since **SCons** is configured to use the **Boost.Python** tool installed with **AutoWIG** that can be used to generate wrappers (see the `../git/STL/src/cpp/SConscript` file).

In [ ]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\SConscript
else:
    !pygmentize ../git/STL/src/cpp/SConscript

The controller is registered in the `../git/STL/src/cpp/AutoWIG.py` file

In [ ]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\AutoWIG.py
else:
    !pygmentize ../git/STL/src/cpp/AutoWIG.py

Then, in addition to the **STL** library, the **StatisKit.STL** library has to be installed in order to have access to some functionalities.
To do so, we use available **Conda** recipes.

In [ ]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\libstatiskit_stl -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/libstatiskit_stl -c statiskit
!conda install -y libstatiskit_stl --use-local -c statiskit --force

As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations (see the `../git/STL/src/cpp/STL.h` file).

In [ ]:
if is_windows:
    !pygmentize ..\git\STL\src\cpp\STL.h
else:
    !pygmentize ../git/STL/src/cpp/STL.h

Once these preliminaries are done, we can proceed to the actual generation of wrappers for the **STL** library.
To do so, we need then to install the *C++* headers. 
This is done using the `cpp` target in **SCons**.

In [ ]:
if is_windows:
    !scons cpp -C ..\git\STL
else:
    !scons cpp -C ../git/STL

Once the headers have been installed in the system, we parse headers with relevant compilation flags.
This is done using the `autowig` target in **SCons**.

In [ ]:
if is_windows:
    !scons autowig -C ..\git\STL
else:
    !scons autowig -C ../git/STL

Here is the list of the generated wrappers (untracked files).

In [ ]:
if is_windows:
    !cd ..\git\STL & git status
else:
    !cd ../git/STL && git status

And here, we present the wrappers generated for the `std::vector< int >` class.

In [ ]:
if is_windows:
    !pygmentize ..\git\STL\src\py\wrapper\wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp
else:
    !pygmentize ../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [ ]:
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\git\STL\bin\conda\python-statiskit_stl -c statiskit -c conda-forge
else:
    !conda build --python=$PYTHON_VERSION ../git/STL/bin/conda/python-statiskit_stl -c statiskit -c conda-forge
!conda install -y python-statiskit_stl --use-local -c statiskit -c conda-forge --force

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [ ]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
VectorInt([0, 1])

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SCONSFLAGS'].lstrip('--site-dir='),
                       'site_autowig',
                       'ASG',
                       'statiskit_stl.pkl'), 'rb') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)